# 使用opencv检测人

# 注意事项：


在训练的时候，我们的正负样本图像默认大小都应该是128×64的，然后提取样本图像的HOG特征，也就是3780维度的特征向量，送入到SVM进行训练，最终的目的就是得到这3781维度的检测器。

在测试的时，检测窗口(大小为128×64)在整个图像的所有位置和尺度上进行扫描，然后提取提取每一个窗口的HOG特征，送入检测器进行判别，最后还需要对输出的金字塔进行非极大值抑制。例如：这里有张图是720×475的，我们选200×100大小的patch，把这个patch从图片里面抠出来，然后再把大小调整成128×64，计算HOG特征，并送入检测器判别是否包含目标。

In [3]:
'''
HOG检测人
'''
import  cv2
import numpy as np

In [4]:
def is_inside(o,i):
    '''
    判断矩形o是不是在i矩形中
    
    args:
        o：矩形o  (x,y,w,h)
        i：矩形i  (x,y,w,h)
    '''
    ox,oy,ow,oh = o
    ix,iy,iw,ih = i
    
    return ox > ix and oy > iy and ox+ow < ix+iw and oy+oh < iy+ih

In [5]:
def draw_person(img,person):
    '''
    在img图像上绘制矩形框person
    
    args:
        img：图像img
        person：人所在的边框位置 (x,y,w,h)
    '''
    x,y,w,h = person
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)

In [6]:
def detect_test(image_path):
    '''
    检测人
    '''
    img = cv2.imread(image_path)
    rows,cols = img.shape[:2]
    sacle = 1.0
    #print('img',img.shape)
    img = cv2.resize(img,dsize=(int(cols*sacle),int(rows*sacle)))
    #print('img',img.shape)
    
    #创建HOG描述符对象
    #计算一个检测窗口特征向量维度：(64/8 - 1)*(128/8 - 1)*4*9 = 3780
    '''
    winSize = (64,128)
    blockSize = (16,16)    
    blockStride = (8,8)
    cellSize = (8,8)
    nbins = 9    
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)  
    '''
    hog = cv2.HOGDescriptor()  
    #hist = hog.compute(img[0:128,0:64])   计算一个检测窗口的维度
    #print(hist.shape)
    detector = cv2.HOGDescriptor_getDefaultPeopleDetector()
    print('detector',type(detector),detector.shape)    
    hog.setSVMDetector(detector)

    
    
    #多尺度检测，found是一个数组，每一个元素都是对应一个矩形，即检测到的目标框
    found,w = hog.detectMultiScale(img)
    print('found',type(found),found.shape)
    
    #过滤一些矩形，如果矩形o在矩形i中，则过滤掉o
    found_filtered = []
    for ri,r in enumerate(found):
        for qi,q in enumerate(found):
            #r在q内？
            if ri != qi and is_inside(r,q):
                break
        else:
            found_filtered.append(r)
            
    for person in found_filtered:
        draw_person(img,person)
        
    cv2.imshow('img',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
image_path = 'person.jpg'
detect_test(image_path)

detector <class 'numpy.ndarray'> (3781, 1)
found <class 'numpy.ndarray'> (6, 4)
